# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [23]:
from collections import Counter

In [24]:
def get_ngrams(input_list, n):
    """
    get all ngrams of size n from the list of strings

    Parmas:
        input_list: list[str] - splitted sentence
        n: ngram size

    Returns:
        list[tuple[str, ..., str]] - list of tuples, each tuple is a ngram of size n
    """
    return list(zip(*[input_list[i:] for i in range(n)]))

In [25]:
class Ngram_model:
    """
    Class represents ngram model.
    It stores frequencies for ngrams starting from size 1 to ngram_size(user_specified).
    It stores set of unique words from provided file with ngrams.
    Also, this class provides a method to calculate MLE of ngram of size <= gram_size(user_specified)

    Params:
        ngram_size: int - size of main ngrams.
        path: str - path to file.
        ngram_freqs

    """

    def __init__(self, ngram_size, path) -> None:
        """
        Params:
            ngram_size: int - should math the size of ngrams in the file.
            path: str - path to the file
            all_ngrams_freqs: dict[int:Counter] - for key=i stores frequencies of sub-ngrams of size i (1 <= i <= ngram_size)
            words: set[str] - unique words.
        """
        self.ngram_size = ngram_size
        self.path = path
        self.all_ngrams_freqs = {i: Counter() for i in range(1, ngram_size+1)}
        self.words = set()
        self._get_text_coca()

    def _get_text_coca(self):
        """
        Read the file with ngrams from coca.
        """
        ngram_counter = 0
        try:
            with open(self.path, 'r', encoding='cp1251') as f:
                for l in f.readlines():
                    words = l.split()
                    words = [w.lower() for w in words]
                    assert len(words) == self.ngram_size + 1, 'check the size of ngram and recreate model'
                    # all sub ngrams share given frequency
                    freq = int(words[0])

                    # collect sub ngrams and main ngram
                    for i in range(1, self.ngram_size+1):
                        ngrams = get_ngrams(words[1:], i)
                        for n in ngrams:
                            self.all_ngrams_freqs[i][n] += freq

                    # collect unique words   
                    for w in words[1:]:
                        self.words.add(w)
        except Exception as e:
            print('check coca dataset file')
            print(e)
            return

    def _count_norm(self, ngram):
        """
        If ngram is (W,W_i), then norm = Count((W,W_k)), where (W,W_k) is every known ngram with context W
        """
        norm = 0
        ngram_len = len(ngram) - 1 if len(ngram) > 1 else 1
        freqs = self.all_ngrams_freqs[ngram_len]
        for k,v in freqs.items():
            if ngram[:ngram_len] == k:
                norm += v
        return norm
    
    def _count(self, ngram):
        """
        If ngram is (W,W_i), then count =  Count((W,W_i))
        """
        counter = 0
        freqs = self.all_ngrams_freqs[len(ngram)]
        for k, v in freqs.items():
            if ngram == k:
                counter += v
        return counter

    def get_mle(self, ngram):
        """
            Calculate the MLE of ngram
        """
        assert len(ngram) <= self.ngram_size, 'provided ngram is to big'
        if len(ngram) == 1:
            norm = len(self.words)
        else:
            norm = self._count_norm(ngram)
        if norm == 0:
            return 0
        count = self._count(ngram)
        return count / norm

In [26]:
class Spell_checker:
    """
    Class implementation for spell checking.
    The main method is `correct`, which checks and tries to correct every word in given string.
    Words are simply diveded by spaces.

    Params:
        ngram_model: Ngram_model to estimate probabilities of possible corrections.
    """

    def __init__(self, ngram_model: Ngram_model) -> None:
        self.ngram_model = ngram_model

    def correct(self, phrase:str, raw=False):
        base, sentence = self._get_list_to_check(phrase)

        # process starting word
        # so starting word is just unigram
        corrected = self._get_best((base[0], ))[-1]
        context = [corrected]
        corrected_phrase = [corrected]

        # extend our context and create sub-ngrams
        # till ngram_size will be reached or the sentences come to the end.
        for w in base[1:]:
            ngram = tuple(context + [w])
            corrected = self._get_best(ngram)[-1]
            corrected_phrase.append(corrected)
            context.append(corrected) 
        
        # then just slide the context window for ngram.
        # so we can utilise corrected context.
        for w in sentence:
            ngram = tuple(context + [w])
            corrected = self._get_best(ngram)[-1]
            corrected_phrase.append(corrected)
            context = context[1:] + [corrected]

        if raw:
            return corrected_phrase
        else:
            res = " ".join([w[0] for w,p in corrected_phrase])
            return res

    def _edits1_en(self, word):
        """
        All edits that are one edit away from `word`.
        
        The idea was taken from Norvig's solution`
        """
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)
    
    def _edits2_en(self, word): 
        """
        All edits that are two edit away from `word`.
        
        The idea was taken from Norvig's solution`
        """
        return (e2 for e1 in self._edits1_en(word) for e2 in self._edits1_en(e1))

    def _candidates(self, ngram, edit_distance):
        """
        Get possible corrections for the last word of ngram based on possible edits.
        Also we check that suggested corrections is in the set of words of ngram model.
        """
        word = ngram[-1]
        if edit_distance == 1:
            candids = set(w for w in self._edits1_en(word) if w in self.ngram_model.words)
        elif edit_distance == 2:
            candids  = set(e2 for e1 in self._edits1_en(word) for e2 in self._edits1_en(e1))
        else:
            return []

        candids_ngrams = []
        for w in candids:
            candids_ngrams.append(tuple(list(ngram[:-1]) + [w]))

        return candids_ngrams
    
    def _get_list_to_check(self, phrase:str):
        """
        Parse given sentence in suitable format
        
        Returns:
            base: list[str] - list of starting words, len(base) < ngram_size
            sentence: list[str] - tail
        """
        sentence = phrase.split()
        i = 0
        base = []

        while i < len(sentence) and i < self.ngram_model.ngram_size - 1:
            base.append(sentence[i])
            i += 1

        return base, sentence[i:]
    
    def _get_best(self, ngram, n=1):
        """
        Method to choose best correction options, sorrted in probability descending order.
        The method utilises idea of backoff for ngram model 
        """
        # if we know such n-gram or n-gram with smaller size, then we return it.
        # so here we use the idea of backoff for ngram model
        for i in range(len(ngram)):
            if self.ngram_model._count(ngram[i:]) > 0:
                return [(ngram[i:], -1)]
        
        # if we did not succeed in the previous stage.
        # we consider the corrections from edit distance = 1
        # and also combine it with backoff
        ngrams_edit_dist1 = self._candidates(ngram, 1)
        pairs = [(ngram, -1)]
        for ngrm in ngrams_edit_dist1:
             for i in range(len(ngrm)):
                 prob = self.ngram_model.get_mle(ngrm[i:])
                 pairs.append((ngrm[i:], prob))

                 # if ngram was found, no need to continue backoff
                 if prob > 0:
                     break

        sorted_pairs = sorted(pairs, key=lambda x: x[1], reverse=True)
        return sorted_pairs[:n]
    


In [27]:
test_model = Ngram_model(ngram_size=2, path='bigrams.txt')
test_model.all_ngrams_freqs

{1: Counter({('a',): 17210595,
          ('aaa',): 924,
          ('all',): 1777377,
          ('an',): 2432831,
          ('and',): 17856270,
          ('another',): 370572,
          ('at',): 3467977,
          ('b',): 11573,
          ('b+',): 45,
          ('b-17',): 58,
          ('b-2',): 571,
          ('b-52',): 287,
          ('b-movie',): 39,
          ('b-plus',): 40,
          ('b.a',): 499,
          ('b.f.a',): 64,
          ('b.s',): 266,
          ('ba',): 294,
          ('babble',): 282,
          ('babbling',): 238,
          ('babe',): 606,
          ('baboon',): 159,
          ('baby',): 70193,
          ('baby-faced',): 31,
          ('baby-sitter',): 243,
          ('babysitter',): 507,
          ('babysitting',): 113,
          ('baccalaureate',): 629,
          ('bach',): 1112,
          ('bachelor',): 3053,
          ('bachelorette',): 154,
          ('bachelors',): 351,
          ('back',): 862478,
          ('back-and-forth',): 189,
          ('back-door',): 

In [28]:
test_spell_checker = Spell_checker(test_model)

In [37]:
test_spell_checker.correct('a weathr was goot and sunn wos shinng', raw=True)

[(('a',), -1),
 (('weather',), 0.4585032032615026),
 (('was',), -1),
 (('good',), 8.657032615026209),
 (('and',), -1),
 (('sun',), 0.7127111240535818),
 (('was',), 82.51292952824694),
 (('shining',), 0.08887594641817123)]

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

The main feature which I decieded to use is a backoff for ngram model. This technique helps to deal with OOV ngrams and balance the influence of context on possible corrections. Also, it is important to notice, that my corrections are made on updated context, that means the model takes into account already made corrections.

My decision model for picking best correction is the following:

- As input we an ngram of size <= ngram_size.
- We check if we know the ngram or its sub-ngrams. if yes, we return last word of ginve subngram
    - if yes, we return last word of ginve subngram
    - if no, we consider possible corrections of edit distance 1

- Here we also use the idea of backoff. We consider subngrams of possible corrections.
    - If such sub-ngram get non 0 mle we stop the process of backoff and continue with possible ngrams

Speaking about the size of ngrams to choose, there is a trade off between speed of inference and ability to capture the context. I was trying 2 and 5 grams, but I think better to use something in the middle, for example 3-grams

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [30]:
# Your code here

In [35]:
test_spell_checker.correct('a weathr was goot and sunn wos shinng', raw=False)

'a weather was good and sun was shining'